In [20]:
import librosa
import librosa.display
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn import preprocessing
import keras
from skimage.transform import rescale, resize, downscale_local_mean

In [118]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(
    physical_devices[0], enable=True
)


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [126]:
path = './data/preprocessed/'
datapartitions = ['ff1010bird_' + str(i+1) for i in range(10)] + ['warblrb10k_public_' + str(i+1) for i in range(10)]
preprocessors = ['_specs', '_mfcc', '_joint']
data = []
labels = []
for partition in datapartitions:
    data.append(np.load(path + partition + preprocessors[2] + '.npy'))
    labels.append(np.load(path + partition + '_labels.npy'))
    print(f'{partition} ready')

data = np.concatenate(data)#[:,:,:,np.newaxis]
labels = np.concatenate(labels).astype(int)

ff1010bird_1 ready
ff1010bird_2 ready
ff1010bird_3 ready
ff1010bird_4 ready
ff1010bird_5 ready
ff1010bird_6 ready
ff1010bird_7 ready
ff1010bird_8 ready
ff1010bird_9 ready
ff1010bird_10 ready
warblrb10k_public_1 ready
warblrb10k_public_2 ready
warblrb10k_public_3 ready
warblrb10k_public_4 ready
warblrb10k_public_5 ready
warblrb10k_public_6 ready
warblrb10k_public_7 ready
warblrb10k_public_8 ready
warblrb10k_public_9 ready
warblrb10k_public_10 ready


In [139]:
model = Sequential([
    keras.Input(shape = (32,650,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters = 16, kernel_size = (3,3), strides = (1,1), activation='relu',padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.SpatialDropout2D(0.1),
    keras.layers.MaxPooling2D(pool_size=(2,2)),

    keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (1,1), activation='relu',padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.SpatialDropout2D(0.1),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    keras.layers.Conv2D(filters = 64, kernel_size = (6,6), strides = (1,1), activation='relu',padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.SpatialDropout2D(0.1),
    keras.layers.MaxPooling2D(pool_size=(2,2)),

    keras.layers.Conv2D(filters = 128, kernel_size = (6,6), strides = (1,1), activation='relu',padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.SpatialDropout2D(0.1),
    keras.layers.GlobalAveragePooling2D(),

    keras.layers.Flatten(),

    keras.layers.Dense(1, activation='sigmoid')
])

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.0001,
    decay_steps=1000,
    decay_rate=0.97
)

Callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        min_delta=0.001,
        patience=50,
        mode='auto',
        restore_best_weights=True
        )
]
optimizer_adam=keras.optimizers.Adam(learning_rate=lr_schedule)
                                     
model.compile(optimizer=optimizer_adam,
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [141]:
start_time = time.time()
model.fit(data, labels, batch_size = 512, epochs=1000, verbose=True, validation_split = 0.1, callbacks=Callbacks)

#model.save('./models/')

print("Training time %s minutes" % ((time.time() - start_time)/60))

Epoch 1/1000
28/28 [==============================] - 9s 310ms/step - loss: 0.5689 - accuracy: 0.8680 - val_loss: 0.4771 - val_accuracy: 0.8124
Epoch 2/1000
28/28 [==============================] - 9s 307ms/step - loss: 0.5681 - accuracy: 0.8675 - val_loss: 0.4656 - val_accuracy: 0.8596
Epoch 3/1000
28/28 [==============================] - 9s 308ms/step - loss: 0.5669 - accuracy: 0.8712 - val_loss: 0.4687 - val_accuracy: 0.8488
Epoch 4/1000
28/28 [==============================] - 9s 308ms/step - loss: 0.5666 - accuracy: 0.8719 - val_loss: 0.4690 - val_accuracy: 0.8628
Epoch 5/1000
28/28 [==============================] - 9s 309ms/step - loss: 0.5676 - accuracy: 0.8692 - val_loss: 0.4789 - val_accuracy: 0.8066
Epoch 6/1000
28/28 [==============================] - 9s 309ms/step - loss: 0.5651 - accuracy: 0.8735 - val_loss: 0.4711 - val_accuracy: 0.8328
Epoch 7/1000
28/28 [==============================] - 9s 309ms/step - loss: 0.5667 - accuracy: 0.8696 - val_loss: 0.4650 - val_accuracy:

Epoch 56/1000
28/28 [==============================] - 9s 309ms/step - loss: 0.5483 - accuracy: 0.9133 - val_loss: 0.4694 - val_accuracy: 0.8430
Epoch 57/1000
28/28 [==============================] - 9s 308ms/step - loss: 0.5488 - accuracy: 0.9117 - val_loss: 0.4658 - val_accuracy: 0.8571
Epoch 58/1000
28/28 [==============================] - 9s 310ms/step - loss: 0.5485 - accuracy: 0.9129 - val_loss: 0.4658 - val_accuracy: 0.8666
Epoch 59/1000
28/28 [==============================] - 9s 312ms/step - loss: 0.5483 - accuracy: 0.9147 - val_loss: 0.4704 - val_accuracy: 0.8309
Epoch 60/1000
28/28 [==============================] - 9s 315ms/step - loss: 0.5499 - accuracy: 0.9100 - val_loss: 0.4656 - val_accuracy: 0.8475
Epoch 61/1000
28/28 [==============================] - 9s 313ms/step - loss: 0.5475 - accuracy: 0.9146 - val_loss: 0.4649 - val_accuracy: 0.8494
Epoch 62/1000
28/28 [==============================] - 9s 312ms/step - loss: 0.5474 - accuracy: 0.9158 - val_loss: 0.4665 - val_ac

28/28 [==============================] - 9s 306ms/step - loss: 0.5327 - accuracy: 0.9497 - val_loss: 0.4653 - val_accuracy: 0.8564
Epoch 169/1000
28/28 [==============================] - 9s 308ms/step - loss: 0.5323 - accuracy: 0.9505 - val_loss: 0.4639 - val_accuracy: 0.8641
Epoch 170/1000
28/28 [==============================] - 9s 307ms/step - loss: 0.5329 - accuracy: 0.9493 - val_loss: 0.4643 - val_accuracy: 0.8628
Epoch 171/1000
28/28 [==============================] - 9s 309ms/step - loss: 0.5319 - accuracy: 0.9516 - val_loss: 0.4636 - val_accuracy: 0.8647
Epoch 172/1000
28/28 [==============================] - 9s 307ms/step - loss: 0.5320 - accuracy: 0.9511 - val_loss: 0.4653 - val_accuracy: 0.8628
Epoch 173/1000
28/28 [==============================] - 9s 310ms/step - loss: 0.5325 - accuracy: 0.9503 - val_loss: 0.4667 - val_accuracy: 0.8590
Epoch 174/1000
28/28 [==============================] - 9s 307ms/step - loss: 0.5324 - accuracy: 0.9506 - val_loss: 0.4663 - val_accuracy: 

In [142]:
model.save('./models/samuli2')

INFO:tensorflow:Assets written to: ./models/samuli2\assets
